In [3]:
import pandas as pd

def normalizar_nombres(nombres):
    nombres = nombres.lower().replace(" ", "_") 
    nombres = nombres.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ü', 'u').replace('ú', 'u')
    nombres = nombres.replace('Á', 'a').replace('É', 'e').replace('Í', 'i').replace('Ó', 'o').replace('Ú', 'u')
    return nombres

def limpiar_columnas(df):
    df.columns = [normalizar_nombres(col) for col in df.columns]
    
    #Columna precio
    df['precio'] = df['precio'].apply(lambda x: float(x.replace(' €/mes', '').replace('€', '').replace(' ', '').replace('.', '').replace(',', '.')) if 'A consultar' not in x else None)
    df['precio'] = df['precio'].astype(float) 

    #Columna precio_m2
    df['precio_m2'] = df['precio_m2'].apply(lambda x: float(x.replace(' €/m²', '').replace('€', '').replace(' ', '').replace('.', '').replace(',', '.')) if 'Error al procesar precio m²' not in x else None)
    df['precio_m2'] = df['precio_m2'].astype(float) 

    #Columna superficie
    df['superficie'] = df['superficie'].apply(lambda x: float(x.replace(' m²', '').replace(' ', '')) if x.replace(' m²', '').replace(' ', '').isnumeric() else None)
    df['superficie'] = df['superficie'].astype(float) 

    #Columna superficie_util
    df['superficie_util'] = df['superficie_util'].apply(lambda x: float(x.replace(' m²', '').replace(' ', '')) if isinstance(x, str) and x.replace(' m²', '').replace(' ', '').isnumeric() else None)
    df['superficie_util'] = df['superficie_util'].astype(float) 

    #Columna superficie_solar
    df['superficie_solar'] = df['superficie_solar'].apply(lambda x: float(x.replace(' m²', '').replace(' ', '').replace('.', '').replace(',', '.')) if isinstance(x, str) and x.replace(' m²', '').replace(' ', '').replace('.', '').replace(',', '').isnumeric() else None)
    df['superficie_solar'] = df['superficie_solar'].astype(float) 

    #Columna superficie_construida
    df['superficie_construida'] = df['superficie_construida'].apply(lambda x: float(x.replace(' m²', '').replace(' ', '')) if isinstance(x, str) and x.replace(' m²', '').replace(' ', '').isnumeric() else None)
    df['superficie_construida'] = df['superficie_construida'].astype(float) 

    # Convertir las columnas a datetime o entero
    df['actualizacion'] = df['actualizacion'].str.extract(r'(\d{2}/\d{2}/\d{4})')
    df['actualizacion'] = pd.to_datetime(df['actualizacion'], format='%d/%m/%Y', errors='coerce')
    df['timestamp'] = pd.to_datetime(df['actualizacion'], errors='coerce')
    df['codigo_postal'] = df['codigo_postal'].astype('Int64')
    df['baños'] = df['baños'].astype('Int64')
    df['habitaciones'] = df['habitaciones'].astype('Int64')
    
    # Columna 'planta'
    df['planta'] = df['planta'].str.replace('ª', '', regex=False).str.strip()
    df['planta'] = df['planta'].replace({'Bajo': 0, 'Principal': 0, 'Semisótano': -1, 'Entresuelo': -1, 'Sótano': -1, 'Más de 20': 21})
    df['planta'] = df['planta'].astype('float64').astype('Int64')




    # Columnas conflictivas
    columnas_conflictivas = ['luz', 'sistema_de_seguridad', 'vidrios_dobles', 
                             'se_aceptan_mascotas', 'carpinteria_interior', 'calefaccion', 'jardin',
                             'trastero', 'calle_asfaltada', 'cocina_equipada', 'agua', 
                             'aire_acondicionado', 'terraza', 'armarios_empotrados', 'balcon', 
                             'gastos_de_comunidad', 'urbanizado', 'garaje', 'puerta_blindada', 
                             'adaptado_a_personas_con_movilidad_reducida', 'ascensor', 'alcantarillado', 
                             'portero_automatico', 'exterior', 'soleado', 'piscina', 'calle_alumbrada', 
                             'lavadero', 'amueblado', 'interior', 'gas', 'tipo_suelo', 
                             'carpinteria_exterior', 'comedor', 'chimenea', 'no_se_aceptan_mascotas', 'telefono']
    
    def limpiar_columnas_conflictivas(valor):
        if pd.isna(valor) or valor.strip() == "":
            return valor 
        elif "no" in valor.lower().split():  
            return "No"
        else:
            return "Si"
    
    for col in columnas_conflictivas:
        df[col] = df[col].apply(limpiar_columnas_conflictivas)
        df[col] = df[col].astype('category')  # Convertir a tipo 'category'
    
    # Eliminar duplicados y referencia
    df = df.drop_duplicates(subset='identificador', keep='first')
    df = df.drop(columns=['referencia'])
    
    # Columna 'se_aceptan_mascotas' 
    df['se_aceptan_mascotas'] = df.apply(lambda row: 'Si' if row['se_aceptan_mascotas'] == 'Si' else 
                                         ('No' if row['no_se_aceptan_mascotas'] == 'Si' else 'N/A'), axis=1)
    # Convertirla en category de Si, No y N/A uniendo y borrando no_se_aceptan_mascotas
    df['se_aceptan_mascotas'] = df['se_aceptan_mascotas'].astype('category')
    df = df.drop(columns=['no_se_aceptan_mascotas'])
    
    # Columna orientacion
    mapeo_direcciones = {
        'norte': 'NORTE', 'n': 'NORTE', 'sur': 'SUR', 's': 'SUR', 
        'este': 'ESTE', 'e': 'ESTE', 'oeste': 'OESTE', 'o': 'OESTE', 
        'suroeste': 'SUROESTE', 'so': 'SUROESTE', 'sureste': 'SURESTE', 
        'se': 'SURESTE', 'noreste': 'NORESTE', 'ne': 'NORESTE', 
        'noroeste': 'NOROESTE', 'no': 'NOROESTE', 'este-sur': 'SURESTE', 
        'sur-oeste': 'SUROESTE', 'norte|este': 'NORESTE', 'norte|oeste': 'NOROESTE',
        'sur|este': 'SURESTE', 'north/east': 'NORESTE', 'north/west': 'NOROESTE',
        'south/east': 'SURESTE', 'south/west': 'SUROESTE', 'sur|oeste': 'SUROESTE',
        'norte|este|oeste': 'NORTE', 'se': 'SURESTE', 'sur, este, oeste': 'SUR',
        'norte::este': 'NORESTE', 'norte::oeste': 'NOROESTE', 'sur::este': 'SURESTE', 
        'sur::oeste': 'SUROESTE', 'sur::este::oeste': 'SUR'
    }
    df['orientacion'] = df['orientacion'].str.lower().map(mapeo_direcciones).fillna('SIN ESPECIFICAR')
    
    return df

# Aplicar la limpieza de columnas al DataFrame
df = pd.read_csv("alquileres_scrap_completo.csv")
dfalquileres = limpiar_columnas(df)
dfalquileres.to_csv("alquilereslimpio.csv", index=False)

In [4]:
dfalquileres.columns

Index(['luz', 'jardin', 'sistema_de_seguridad', 'conservacion',
       'vidrios_dobles', 'se_aceptan_mascotas', 'timestamp',
       'carpinteria_interior', 'calefaccion', 'trastero', 'href',
       'calle_asfaltada', 'tipo_de_casa', 'cocina_equipada', 'agua',
       'emisiones', 'aire_acondicionado', 'terraza', 'armarios_empotrados',
       'actualizacion', 'superficie', 'balcon', 'gastos_de_comunidad',
       'urbanizado', 'superficie_construida', 'garaje', 'consumo',
       'puerta_blindada', 'adaptado_a_personas_con_movilidad_reducida',
       'identificador', 'ascensor', 'superficie_util', 'nombre',
       'alcantarillado', 'planta', 'baños', 'portero_automatico', 'exterior',
       'soleado', 'piscina', 'calle_alumbrada', 'precio_m2', 'precio',
       'lavadero', 'orientacion', 'amueblado', 'interior', 'habitaciones',
       'gas', 'tipo_suelo', 'carpinteria_exterior', 'superficie_solar',
       'comedor', 'antiguedad', 'codigo_postal', 'ubicacion', 'agencia',
       'chimenea', '

In [5]:
for col in dfalquileres.columns:
    print(f"Valores únicos para la columna {col}:")
    print(dfalquileres[col].unique())
    print("-" * 40)

Valores únicos para la columna luz:
[NaN, 'Si']
Categories (1, object): ['Si']
----------------------------------------
Valores únicos para la columna jardin:
[NaN, 'Si']
Categories (1, object): ['Si']
----------------------------------------
Valores únicos para la columna sistema_de_seguridad:
[NaN, 'Si']
Categories (1, object): ['Si']
----------------------------------------
Valores únicos para la columna conservacion:
[nan 'En buen estado' 'Reformado' 'A estrenar' 'A reformar']
----------------------------------------
Valores únicos para la columna vidrios_dobles:
[NaN, 'Si', 'No']
Categories (2, object): ['No', 'Si']
----------------------------------------
Valores únicos para la columna se_aceptan_mascotas:
['N/A', 'No', 'Si']
Categories (3, object): ['N/A', 'No', 'Si']
----------------------------------------
Valores únicos para la columna timestamp:
<DatetimeArray>
['2025-02-21 00:00:00', '2025-02-20 00:00:00', '2025-02-13 00:00:00',
 '2025-02-07 00:00:00', '2025-02-11 00:00:00'

In [6]:
dfalquileres.dtypes

luz                                                 category
jardin                                              category
sistema_de_seguridad                                category
conservacion                                          object
vidrios_dobles                                      category
se_aceptan_mascotas                                 category
timestamp                                     datetime64[ns]
carpinteria_interior                                category
calefaccion                                         category
trastero                                            category
href                                                  object
calle_asfaltada                                     category
tipo_de_casa                                          object
cocina_equipada                                     category
agua                                                category
emisiones                                             object
aire_acondicionado      